# Self Attention

### 3.3 Attending to different parts of the input with self attention

#### 3.3.1 A simple self-attention mechanism without trainable weights

In [25]:
# consider that z^2 is the input vector x^2 but with information about all the other input
import torch

inputs = torch.tensor(
    [
        [0.43, 0.15, 0.89],  # Your     (x^1)
        [0.55, 0.87, 0.66],  # journey  (x^2)
        [0.57, 0.85, 0.64],  # starts   (x^3)
        [0.22, 0.58, 0.33],  # with     (x^4)
        [0.77, 0.25, 0.10],  # one      (x^5)
        [0.05, 0.80, 0.55],  # step     (x^6)
    ]
) # rows are words, columns are embedding dimensions
inputs

tensor([[0.4300, 0.1500, 0.8900],
        [0.5500, 0.8700, 0.6600],
        [0.5700, 0.8500, 0.6400],
        [0.2200, 0.5800, 0.3300],
        [0.7700, 0.2500, 0.1000],
        [0.0500, 0.8000, 0.5500]])

In [26]:
# Suppose we use the second input as query => q^2 = x^2
# The attention scores are: w21 = x^1 @ q^2.T; w22 = x^2 @ q^2.T; ... (w = omega)
# The attention weights are then these values normalized via softmax
query_2 = inputs[1]
score_dim = inputs.shape[0]
attn_scores_2 = torch.zeros(score_dim)
for i, input_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(input_i, query_2) # Multiplying each input with input_2 to get the attention scores for input_2
print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [27]:
query_2.shape

torch.Size([3])

In [28]:
# The above is equivalent to a matrix multiplication:
assert torch.equal(attn_scores_2, inputs @ query_2)

In [29]:
# Step 2: Normalize the scores so they sum up to 1 (they are thus called "weights")
# This is done to ease gradient learning, avoiding too large/small values
# Softmax does this best, by accounting for outliers
attn_weights_2 = torch.softmax(attn_scores_2, dim=0) # it's a just one row anyway 
attn_weights_2

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

In [30]:
print(inputs.shape)
print(attn_weights_2.shape)

torch.Size([6, 3])
torch.Size([6])


In [31]:
# Step 3: compute the context vector z_2 by multiplying the inputs by their respective weights
# Which is again just a matrix dot product
context_vec_2 = attn_weights_2 @ inputs
context_vec_2

tensor([0.4419, 0.6515, 0.5683])

#### 3.3.2 Computing attention weights for ALL input tokens

In [32]:
attn_scores_all = torch.empty(6,6)

# Apply previous Step 1 to all pairwise elements to compute the unnormalized attention score matrix
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores_all[i, j] = x_i @ x_j

# Equivalent to:
# attn_scores_all = inputs @ inputs.T

print(attn_scores_all)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [33]:
print(inputs)
print()
print(inputs.T)

tensor([[0.4300, 0.1500, 0.8900],
        [0.5500, 0.8700, 0.6600],
        [0.5700, 0.8500, 0.6400],
        [0.2200, 0.5800, 0.3300],
        [0.7700, 0.2500, 0.1000],
        [0.0500, 0.8000, 0.5500]])

tensor([[0.4300, 0.5500, 0.5700, 0.2200, 0.7700, 0.0500],
        [0.1500, 0.8700, 0.8500, 0.5800, 0.2500, 0.8000],
        [0.8900, 0.6600, 0.6400, 0.3300, 0.1000, 0.5500]])


In [34]:
# step 2: normalize
attn_weights_all = torch.softmax(attn_scores_all, dim=1)
print(attn_weights_all)
attn_weights_all.sum(dim=-1)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])

In [35]:
# Step 3: Get all context vectors z_i
context_vec_all = attn_weights_all @ inputs
print(context_vec_all)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


These are now the same input tokens, modified to take context (surrounding words) into account.

# 3.4 Implementing self-attention with trainable weights (_fucked_)

A.k.a _scaled dot-product attention_

####  3.4.1 Implementing the weights step by step
We start by calculating the context vector for just one input element, x_2

In [36]:
x_2 = inputs[1]
d_in = inputs.shape[1] # input's embedding size, d=3
d_out = 2 # output's embedding size, d=2
# usually d_in==d_out, but we change this for better illustration

# Set the query, key, and value weight matrices
# The purpose of these weight matrices is to project each input into things that are trainable and usable in different scenarios (when input is considered the query, for example) 
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False) # This should be True, but we turn it off to reduce output clutter
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

# Get q,k,v vecs
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value 

print(query_2) # 2d, since d_out=2

tensor([0.4306, 1.4551])


In [37]:
# Even though we're only trying to get z_2, we still need all keys and values (for the other inputs)
# because they're involved in computing the attention weights wrt q_2
keys = inputs @ W_key
values = inputs @ W_value
keys.shape

torch.Size([6, 2])

In [38]:
# Step 2: Compute the attention scores
# To do this, you multiply the given query by all the keys
attn_scores_2 = keys @ query_2 # same as doing query2 @ keys.T
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [39]:
# Step 3: Getting attetion weights via scaled softmax
d_k = keys.shape[-1] # 2
attn_weights_2 = torch.softmax(attn_scores_2 / (d_k**0.5), dim=-1) # To prevent grads from getting too large and the softmax function from acting like a step function
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


In [40]:
# Step 4: Context vec for x_2
context_vec_2 = attn_weights_2 @ values #* VALUES!
z_2 = context_vec_2
print(context_vec_2)

tensor([0.3061, 0.8210])


#### 3.4.2 Implementing a Python class for the self-attention module

In [41]:
import torch.nn as nn
from torch import softmax

class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        queries = x @ W_query
        keys = x @ W_key
        values = x @ W_value
        attn_scores = queries @ keys.T # both are 6,2 with example input
        attn_weights = softmax(attn_scores / (keys.shape[-1]**0.5), dim=-1) # omega, or w
        context_vecs = attn_weights @ values
        return context_vecs
        

In [42]:
torch.manual_seed(123)
sa = SelfAttention_v1(d_in, d_out)
print(sa(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]])


In [43]:
# Update the model by using Linear instead of Parameter, which
# has an optimized weight initialization scheme and, when bias=False,
# effectively does a matrix multiplication
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, kqv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=kqv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=kqv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=kqv_bias)

    def forward(self, x):
        queries = x @ W_query
        keys = x @ W_key
        values = x @ W_value
        attn_scores = queries @ keys.T  # both are 6,2 with example input
        attn_weights = softmax(attn_scores / (keys.shape[-1] ** 0.5), dim=-1)  # omega, or w
        context_vecs = attn_weights @ values
        return context_vecs

In [44]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]])
